# Play with the dataset

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/notebook/imgaug/')

import os, numpy, glob, collections, random, h5py, shutil, pandas, \
    time, functools, json, traceback, itertools

from PIL import Image
from IPython.display import display, SVG
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%pylab inline

from sklearn.utils import shuffle
import imgaug.imgaug as ia
from imgaug.imgaug import augmenters as iaa
from imgaug.imgaug import parameters as iap

from prepare_images_utils import *

Populating the interactive namespace from numpy and matplotlib


/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
Using TensorFlow backend.


## Build some statistics

In [3]:
hist_bins = numpy.arange(0, 255.1, 10, dtype=numpy.uint8)
half_bins = hist_bins.shape[0] // 2
def get_image_stat(fname):
    img = Image.open(fname)
    pixels = numpy.asarray(img)
    return numpy.histogram(pixels, bins = hist_bins)[0]


def contains_white(fname):
    img = Image.open(fname)
    pixels = numpy.asarray(img)
    return (pixels > 0).any()


def get_positive_prefixes(dirname, n_jobs=20):
    filenames = glob.glob(os.path.join(dirname, '*_out.png'))
    white_info = Parallel(n_jobs=n_jobs)(delayed(contains_white)(fname)
                                         for fname in filenames)
    return [fname[:-8] for fname, has_white
            in zip(filenames, white_info)
            if has_white]


def copy_only_positive(src_dir, target_dir, n_jobs=20):
    for prefix in get_positive_prefixes(src_dir, n_jobs=n_jobs):
        shutil.copy2(prefix + '_in.png', target_dir)
        shutil.copy2(prefix + '_out.png', target_dir)


def aggregate_stat_for_multiple_images(fnames):
    marginal_hist = numpy.zeros(hist_bins.shape[0] - 1,
                                dtype=numpy.float)
    number_of_white_images = 0
    img_cnt = 0.0
    for cur_hist in Parallel(n_jobs=20)(delayed(get_image_stat)(fname)
                                        for fname in fnames):
        marginal_hist += cur_hist
        img_cnt += 1
        if cur_hist[half_bins:].sum() > 0:
            number_of_white_images += 1
    return marginal_hist / img_cnt, number_of_white_images


def folder_stat(dirname):
    fig, (in_ax, out_ax) = plt.subplots(2)

    in_files = list(glob.glob(os.path.join(dirname, '*_in.png')))
    in_files_cnt = len(in_files)
    in_hist, in_white_cnt = aggregate_stat_for_multiple_images(in_files)
    in_ax.bar(hist_bins[:-1], in_hist, label='In')
    in_ax.legend()
    in_percent = in_white_cnt / in_files_cnt * 100.0
    print(f'In white {in_white_cnt} / {in_files_cnt}, {in_percent}%')

    out_files = glob.glob(os.path.join(dirname, '*_out.png'))
    out_files_cnt = len(out_files)
    out_hist, out_white_cnt = aggregate_stat_for_multiple_images(out_files)
    out_ax.bar(hist_bins[:-1], out_hist, label='Out')
    out_ax.legend()
    out_percent = out_white_cnt / out_files_cnt * 100.0
    print(f'Out white {out_white_cnt} / {out_files_cnt}, {out_percent}%')

In [4]:
# copy_only_positive('./data/5_ready/train/', './data/5_ready/train_only_positive/')
# copy_only_positive('./data/5_ready/val/', './data/5_ready/val_only_positive/')
# copy_only_positive('./data/5_ready/test/', './data/5_ready/test_only_positive/')

In [5]:
# folder_stat('./data/5_ready/train/')

In [6]:
# folder_stat('./data/5_ready/train_only_positive/')

In [7]:
# folder_stat('./data/5_ready/test/')

## Prepare dataset

### Full Train for Production

In [8]:
# convert_directory_to_hdf5('/notebook/data/7_full/train/', '/notebook/data/7_full/train.hdf')
# convert_directory_to_hdf5('/notebook/data/7_full/val/', '/notebook/data/7_full/val.hdf')

### Train-Val

In [9]:
# !rm /notebook/data/5_ready/*.hdf

In [10]:
# %%time
# convert_directory_to_hdf5('/notebook/data/5_ready/train/', '/notebook/data/5_ready/train.hdf')
# convert_directory_to_hdf5('/notebook/data/5_ready/val/', '/notebook/data/5_ready/val.hdf')

### CV

In [11]:
# for fold in glob.glob('/notebook/data/6_eval/*/'):
#     convert_directory_to_hdf5(os.path.join(fold, 'train'), os.path.join(fold, 'train.hdf'))
#     convert_directory_to_hdf5(os.path.join(fold, 'val'), os.path.join(fold, 'val.hdf'))

### Load

In [12]:
# train_h5.close()
# val_h5.close()
# test_h5.close()

In [13]:
# train_h5 = h5py.File('/notebook/data/5_ready/train.hdf', 'r')
# val_h5 = h5py.File('/notebook/data/5_ready/val.hdf', 'r')
# train_h5 = h5py.File('/notebook/data/7_full/train.hdf', 'r')
# val_h5 = h5py.File('/notebook/data/7_full/val.hdf', 'r')

# train_in, train_out = train_h5['in_data'], train_h5['out_data']
# val_in, val_out = val_h5['in_data'], val_h5['out_data']

# train_in, train_out = load_dataset('/notebook/data/5_ready/train/')
# val_in, val_out = load_dataset('/notebook/data/5_ready/val/')
# train_in = (train_in * 255).astype('uint8')
# train_out = (train_out * 255).astype('uint8')
# val_in = (val_in * 255).astype('uint8')
# val_out = (val_out * 255).astype('uint8')
# print(train_in.shape, train_out.shape)
# print(val_in.shape, val_out.shape)
# # print(test_in.shape, test_out.shape)

In [14]:
# sample_i = 25
# display(arr_to_img(train_in[sample_i, :, :, 0]))
# display(arr_to_img(train_out[sample_i, :, :, :3]))

In [15]:
# sample_i = 14
# display(arr_to_img(val_in[sample_i, :, :, 0]))
# display(arr_to_img(val_out[sample_i, :, :, :3]))

In [25]:
def batch_getter(in_dir, batch_size=1):
    while True:
        in_data, out_data = load_dataset(in_dir, n_jobs=6, take_n=batch_size)
        in_data = (in_data * 255).astype('float32')
        out_data = (out_data * 255).astype('float32')
        yield ia.Batch(images=in_data,
                       heatmaps=out_data)

def make_batch_getter(in_dir, batch_size=32):
    def _f():
        yield from batch_getter(in_dir, batch_size=batch_size)
    return _f

def augmented_batch_gen(base_gen, augmenter, heatmaps_hooks=None):
    for batch in base_gen():
        det = augmenter.to_deterministic() if not augmenter.deterministic else augseq
        images_aug = numpy.asarray(det.augment_images(batch.images)).astype('float32') / 255
        heatmaps_aug = numpy.asarray(det.augment_images(batch.heatmaps, hooks=heatmaps_hooks)).astype('float32') / 255
        yield (images_aug, heatmaps_aug)

def make_augmented_batch_gen(base_gen, augmenter, heatmaps_hooks=None):
    def _f():
        yield from augmented_batch_gen(base_gen, augmenter, heatmaps_hooks=heatmaps_hooks)
    return _f

In [17]:
# %%time
# i = iter(make_batch_getter('./data/5_ready/train/')())
# for _ in range(20):
#     qq = next(i)

In [18]:
# print(next(iter(make_augmented_batch_gen(make_batch_getter(train_in, train_out), augmenter)()))[1].shape)
# display(arr_to_img(next(iter(make_augmented_batch_gen(make_batch_getter(train_in, train_out), augmenter)()))[1][0, :, :]))

## Define U-net

## Train-Val-Test

In [26]:
BATCH_SIZE = 4
TRANSFORM_TO_WIDTH = 400
TRANSFORM_TO_HEIGHT = 400
emboss_aug = iaa.Emboss(alpha=0.5, strength=13.5)
window_aug = iaa.Window((TRANSFORM_TO_HEIGHT, TRANSFORM_TO_WIDTH))
train_aug = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
#     iaa.Affine(rotate=iap.DiscreteUniform(0, 3) * 90, cval=(0, 0, 255)),
    iaa.CropAndPad(percent=(-0.10, 0.10), pad_cval=255),
    emboss_aug,
    window_aug
])
val_aug = iaa.Sequential([
    emboss_aug,
    window_aug
])
fake_batch = next(iter(make_augmented_batch_gen(make_batch_getter('/notebook/data/5_ready/train/',
                                                                  batch_size=BATCH_SIZE),
                                                train_aug)()))
# for i in range(fake_batch[0].shape[0]):
#     display(arr_to_img(fake_batch[0][i, :, :, 0]))
#     display(arr_to_img(fake_batch[1][i, :, :]))

In [27]:
model = get_unet_row_col_info(fake_batch[0].shape[1:], fake_batch[1][0].shape[-1])
# model.summary()
# display(SVG(model_to_dot(model).create(prog='dot', format='svg')))

log_dir = os.path.join('.',
                       'tb_logs',
                       datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))

model_file = os.path.join(log_dir, 'model')
# model_file = './full_model'

if os.path.exists(model_file):
    os.remove(model_file)
try:
    history = History()
    model.fit_generator(augmented_batch_gen(make_batch_getter('./data/5_ready/train/',
                                                              batch_size=BATCH_SIZE),
                                            train_aug),
                        BATCH_SIZE * 80,
                        epochs=200,
                        callbacks=[
                                   EarlyStopping(verbose=1, patience=10, monitor='val_dice_coef_01', mode='max'),
                                   ModelCheckpoint(filepath=model_file, verbose=1, monitor='val_dice_coef_01', mode='max', save_best_only=True),
                                   history,
                                   TensorBoard(log_dir=log_dir,
#                                                histogram_freq=1,
                                               batch_size=BATCH_SIZE,
                                               write_graph=True,
                                               write_grads=True,
                                               write_images=True),
                                   ReduceLROnPlateau(factor=0.5,
                                                     patience=40,
                                                     epsilon=1e-3,
                                                     verbose=1,
                                                     cooldown=10,
                                                     min_lr=1e-6)
                        ],
                        validation_data=augmented_batch_gen(make_batch_getter('./data/5_ready/val/',
                                                                              batch_size=BATCH_SIZE),
                                                            val_aug),
                        validation_steps=BATCH_SIZE * 20,
                        verbose=1)
except KeyboardInterrupt:
    print(traceback.format_exc())
    pass
# model = load_model(model_file, custom_objects=dict(dice_coef_loss=dice_coef_loss,
#                                                    dice_coef=dice_coef,
#                                                    dice_ce_loss=dice_ce_loss,
#                                                    dice_coef_0=dice_coef_0,
#                                                    dice_coef_1=dice_coef_1,
#                                                    dice_coef_01=dice_coef_01,
#                                                    dice_ce_01_loss=dice_ce_01_loss))

Epoch 1/200


/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)


 29/320 [=>............................] - ETA: 244s - loss: 2.0701 - dice_coef_0: 0.0446 - dice_coef_1: 0.1664 - dice_coef_01: 0.1055 - categorical_crossentropy: 0.8985

/root/.pyenv/versions/3.6.0/lib/python3.6/site-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


320/320 [==============================] - ETA: 0s - loss: 1.4201 - dice_coef_0: 0.0841 - dice_coef_1: 0.2900 - dice_coef_01: 0.1870 - categorical_crossentropy: 0.5239 - val_loss: 1.5789 - val_dice_coef_0: 0.0355 - val_dice_coef_1: 0.2188 - val_dice_coef_01: 0.1271 - val_categorical_crossentropy: 0.4468
Epoch 2/200
320/320 [==============================] - ETA: 0s - loss: 1.1984 - dice_coef_0: 0.1294 - dice_coef_1: 0.3573 - dice_coef_01: 0.2433 - categorical_crossentropy: 0.4392 - val_loss: 1.1751 - val_dice_coef_0: 0.1145 - val_dice_coef_1: 0.3672 - val_dice_coef_01: 0.2408 - val_categorical_crossentropy: 0.4048
Epoch 3/200
320/320 [==============================] - ETA: 0s - loss: 1.0847 - dice_coef_0: 0.1535 - dice_coef_1: 0.3937 - dice_coef_01: 0.2736 - categorical_crossentropy: 0.4032 - val_loss: 1.1305 - val_dice_coef_0: 0.1333 - val_dice_coef_1: 0.3979 - val_dice_coef_01: 0.2656 - val_categorical_crossentropy: 0.3971
Epoch 4/200
320/320 [==============================] - ETA: 0

320/320 [==============================] - ETA: 0s - loss: 0.8772 - dice_coef_0: 0.2440 - dice_coef_1: 0.4909 - dice_coef_01: 0.3674 - categorical_crossentropy: 0.3500 - val_loss: 0.8457 - val_dice_coef_0: 0.2141 - val_dice_coef_1: 0.4941 - val_dice_coef_01: 0.3541 - val_categorical_crossentropy: 0.2741
Epoch 31/200
320/320 [==============================] - ETA: 0s - loss: 0.8780 - dice_coef_0: 0.2439 - dice_coef_1: 0.4926 - dice_coef_01: 0.3683 - categorical_crossentropy: 0.3487 - val_loss: 1.0989 - val_dice_coef_0: 0.1692 - val_dice_coef_1: 0.3497 - val_dice_coef_01: 0.2595 - val_categorical_crossentropy: 0.3466
Epoch 32/200
320/320 [==============================] - ETA: 0s - loss: 0.8346 - dice_coef_0: 0.2537 - dice_coef_1: 0.5093 - dice_coef_01: 0.3815 - categorical_crossentropy: 0.3268 - val_loss: 0.7462 - val_dice_coef_0: 0.2077 - val_dice_coef_1: 0.5746 - val_dice_coef_01: 0.3912 - val_categorical_crossentropy: 0.2490
Epoch 33/200
320/320 [==============================] - ETA

## Train on CV

In [28]:
# for fold in glob.glob('/notebook/data/6_eval/*/'):
#     fold_train_h5 = h5py.File(os.path.join(fold, 'train.hdf'), 'r')
#     fold_val_h5 = h5py.File(os.path.join(fold, 'val.hdf'), 'r')

#     fold_train_in = fold_train_h5['in_data']
#     fold_train_out = fold_train_h5['out_data']

#     fold_val_in = fold_val_h5['in_data']
#     fold_val_out = fold_val_h5['out_data']

#     fold_model_fname = os.path.join(fold, 'model')
#     if os.path.exists(fold_model_fname):
#         os.remove(fold_model_fname)

#     history = History()
#     model = get_unet(fold_train_in.shape[1:], fold_val_out.shape[-1])
#     model.fit(fold_train_in,
#               fold_train_out,
#               batch_size=8,
#               epochs=20,
#               callbacks=[
#                          EarlyStopping(verbose=1, patience=10, monitor='val_dice_coef_01', mode='max'),
#                          ModelCheckpoint(filepath=fold_model_fname, verbose=1, monitor='val_dice_coef_01', mode='max', save_best_only=True),
#                          history
# #                          ReduceLROnPlateau(factor=0.3, patience=10, epsilon=1e-3, verbose=1, cooldown=5, min_lr=1e-6)
# #                          LearningRateScheduler(lambda epoch: 1e-1 * (0.93 ** epoch))
#                          ],
#               validation_data=[fold_val_in, fold_val_out],
#               shuffle='batch',
#               verbose=1)
#     with open(os.path.join(fold, 'train_history.js'), 'w') as f:
#         json.dump(history.history, f)

## Results Vis

In [34]:
# cv_hist = pandas.concat([pandas.DataFrame(json.load(open(os.path.join(fold, 'train_history.js'), 'r')))
#                          for fold in glob.glob('/notebook/data/6_eval/*/')],
#                         keys=[os.path.basename(fold) for fold in glob.glob('/notebook/data/6_eval/*')],
#                         names=['fold'],
#                         axis=1)
# cv_hist.xs('val_dice_coef_01', level=1, axis=1).plot()

In [35]:
# ax = pandas.DataFrame(history.history)[[k for k in history.history.keys()]].plot(figsize=(16, 10))
# ax.set_ylim(0, 1)

In [ ]:
def make_layer_vis(model, layer_i):
    output_getter = K.function([model.layers[0].input, K.learning_phase()],
                               [model.layers[layer_i].output])
    def _impl(img):
        inp = numpy.expand_dims(numpy.expand_dims(numpy.array(img), -1), 0)
        outp = output_getter([inp, False])[0]
        flat = numpy.rollaxis(outp, -1, 2).reshape((outp.shape[1], outp.shape[2]*outp.shape[-1]))
        display((flat.min(), flat.max(), flat.mean()))
        flat -= flat.min()
        flat /= (flat.max() + 1e-4)
#         flat = outp[0, :, :, 0] #.reshape((outp.shape[1], outp.shape[2]*outp.shape[-1]))
        return Image.fromarray((flat*255).astype('uint8'), mode='L')
    return _impl

def vis_layer(img, model):
    visualizers = [(model.layers[i], make_layer_vis(model, i))
                   for i in (i for i in range(len(model.layers))
                             if not isinstance(model.layers[i],
                                               (Dropout,
                                                BatchNormalization,
                                                )))]
    for layer, v in visualizers:
        display(layer.name)
        display(v(img))
        time.sleep(1.0)

# vis_img = read_images_to_tensor(['./data/5_ready/test_tiny/12147373-0005_2_in.png'])[0]
vis_img = fake_batch[0][2, :, :, 0]
vis_layer(vis_img, model)